In [3]:
#Install Required Libraries
!pip install requests beautifulsoup4 pandas


In [4]:
#Send a Request to LinkedIn Jobs API
import requests
from bs4 import BeautifulSoup

# Define the job search URL (Modify based on your location and job role)
job_role = "data-analyst"
location = "india"
url = f"https://www.linkedin.com/jobs/search/?currentJobId=4186711178&keywords=data%20analyst&origin=SWITCH_SEARCH_VERTICAL"

# Set headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send request
response = requests.get(url, headers=headers)
print("Response Status Code:", response.status_code)

# Check if request was successful
if response.status_code == 200:
    print("Successfully fetched the page!")
else:
    print("Failed to fetch the page. LinkedIn might be blocking requests.")


Response Status Code: 200
Successfully fetched the page!


In [10]:
# Parse HTML response
soup = BeautifulSoup(response.text, "html.parser")

# Extract job postings
jobs = soup.find_all("li")  # Adjust class if needed

job_data = []

for job in jobs:
    title = job.find("h3")
    company = job.find("h4")
    location = job.find("span", class_="job-search-card__location")
    job_link = job.find("a")

    if title and company and location and job_link:
        job_data.append([
            title.text.strip(),
            company.text.strip(),
            location.text.strip(),
            "https://www.linkedin.com" + job_link["href"]
        ])

# Convert to DataFrame
df = pd.DataFrame(job_data, columns=["Job Title", "Company", "Location", "Job Link"])
print(df.head())



         Job Title                Company           Location  \
0     Data Analyst  The Coca-Cola Company        Atlanta, GA   
1     Data Analyst                   Meta       New York, NY   
2  Data Analyst II              Pinterest       New York, NY   
3     Data Analyst                   Meta    Los Angeles, CA   
4  Data Analyst II              Pinterest  San Francisco, CA   

                                            Job Link  
0  https://www.linkedin.comhttps://www.linkedin.c...  
1  https://www.linkedin.comhttps://www.linkedin.c...  
2  https://www.linkedin.comhttps://www.linkedin.c...  
3  https://www.linkedin.comhttps://www.linkedin.c...  
4  https://www.linkedin.comhttps://www.linkedin.c...  


In [6]:
#Save Data to CSV
import pandas as pd

df = pd.DataFrame(job_data, columns=["Job Title", "Company", "Location", "Job Link"])
df.to_csv("linkedin_jobs.csv", index=False)
print("Job Data saved to CSV!")


Job Data saved to CSV!


In [7]:
#Save to SQLite Database
import sqlite3

conn = sqlite3.connect("linkedin_jobs.db")
df.to_sql("jobs", conn, if_exists="replace", index=False)
conn.close()
print("Job Data saved to SQLite Database!")


Job Data saved to SQLite Database!


In [11]:
#Retrieve and display job listings stored in the database
conn = sqlite3.connect("linkedin_jobs.db")
query = "SELECT * FROM jobs LIMIT 5"
df_jobs = pd.read_sql(query, conn)
print(df_jobs)
conn.close()

         Job Title                Company           Location  \
0     Data Analyst  The Coca-Cola Company        Atlanta, GA   
1     Data Analyst                   Meta       New York, NY   
2  Data Analyst II              Pinterest       New York, NY   
3     Data Analyst                   Meta    Los Angeles, CA   
4  Data Analyst II              Pinterest  San Francisco, CA   

                                            Job Link  
0  https://www.linkedin.com/jobs/view/data-analys...  
1  https://www.linkedin.com/jobs/view/data-analys...  
2  https://www.linkedin.com/jobs/view/data-analys...  
3  https://www.linkedin.com/jobs/view/data-analys...  
4  https://www.linkedin.com/jobs/view/data-analys...  
